In [54]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
%matplotlib inline

test_dataset = h5py.File('/Users/v/test_catvnoncat.h5', "r")
train_dataset = h5py.File('/Users/v/train_catvnoncat.h5', "r")

train_X = np.array(train_dataset["train_set_x"][1:166])
train_Y = np.array(train_dataset["train_set_y"][1:166])
test_X = np.array(train_dataset["train_set_x"][167:])
test_Y = np.array(train_dataset["train_set_y"][167:])
classes = np.array(train_dataset["list_classes"][:])

train_Y = train_Y.reshape((1, train_Y.shape[0]))
test_Y  = test_Y.reshape((1, test_Y.shape[0]))

train_X_flat = (train_X.reshape(train_X.shape[0], -1)/255).T
test_X_flat = (test_X.reshape(test_X.shape[0], -1)/255).T


In [55]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [56]:
def propagate(w, b, X, Y):
    m = X.shape[1]

    A = sigmoid(np.dot(w.T,X) + b)
    cost = np.sum(((- np.log(A))*Y + (-np.log(1-A))*(1-Y)))/m 
    dw = (np.dot(X,(A-Y).T))/m
    db = (np.sum(A-Y))/m

    cost = np.squeeze(cost)

    return dw, db, cost

In [57]:
X = train_X_flat
Y = train_Y
dim = train_X_flat.shape[0]
w = np.zeros((dim,1))
b = 0
num_iterations = 10000
learning_rate = 0.006
record_cost = 50 
costs = []

for i in range(num_iterations):
    dw, db, cost = propagate(w, b, X, Y)

    w = w - (learning_rate*dw)
    b = b - (learning_rate*db)

print(w)
print(b)

[[ 0.01668438]
 [-0.04412772]
 [-0.02552509]
 ...
 [-0.02337981]
 [-0.0446268 ]
 [ 0.05443966]]
0.04699355825140062


In [58]:
A = sigmoid(np.dot(w.T,test_X_flat) + b)
Y_predict_test = (A >= 0.5) * 1.0
Y_predict_train = sigmoid(np.dot(w.T,X) + b)

print(f"train accuracy: {(100 - np.mean(np.abs(Y_predict_train - train_Y)) * 100):2f}")
print(f"test accuracy: {(100 - np.mean(np.abs(Y_predict_test - test_Y)) * 100):2f}")

train accuracy: 98.127639
test accuracy: 50.000000


In [59]:
def predict(w, b, X):
    
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    A = sigmoid(np.dot((w.T), X) + b)
    
    for i in range(A.shape[1]):
        if(A[0, i] <= 0.5):
            Y_prediction[0, i] = 0
        else:
            Y_prediction[0, i] = 1
    
    Y_prediction.reshape(1, m)
    
    return Y_prediction

In [68]:
train_results = predict(w, b, train_X_flat)

wrong_pred = []
for i in range(len(train_results)):
    if train_results[i] != train_Y[i]:
        wrong_pred.append[i]

print(wrong_pred)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()